In [114]:
import numpy as np
from PCMM.mixture_EM_loop import mixture_EM_loop
from PCMM.mixture_torch_loop import mixture_torch_loop
from PCMM.helper_functions import calc_NMI
import matplotlib.pyplot as plt

## Tutorial on Phase Coherence Mixture Modeling (PCMM)

When dealing with multivariate narrowband data, such as those from functional neuroimaging modalities such as fMRI, EEG, or MEG, it is useful to model the instantaneous phase coherence between channels as a proxy for instantaneous correlation. This toolbox provides several ways to model such data, though we refer to our paper for all the details: https://www.biorxiv.org/content/10.1101/2024.11.15.623830v1.

In this tutorial, we will demonstrate how to use the toolbox to simulate phase coherence data and fit a K-means, mixture, or HMM. We will use simulated data corresponding to K=2 true components with consistent phase shifts and added noise.

## Generate synthetic data

In [115]:
noise_scale = np.pi*2/3
num_points_per_cluster = 1000
p = 3 # Data dimensionality
q = 2 # Number of frames for grassmann data
K = 2 # Number of true clusters

# True thetas
theta1 = np.array([0,np.pi/2,0]) 
theta2 = np.array([0,0,np.pi/2]) 
thetas = [theta1,theta2]

data_real_projective_hyperplane = np.zeros((num_points_per_cluster,p,K))
data_complex_projective_hyperplane = np.zeros((num_points_per_cluster,p,K),dtype=complex) 
data_grassmann = np.zeros((num_points_per_cluster,p,q,K)) 
data_spsd = np.zeros((num_points_per_cluster,p,q,K))
data_ts = np.zeros((num_points_per_cluster,p,K))

true_labels = np.zeros((K,num_points_per_cluster*2))
true_labels[0,num_points_per_cluster:] = 1
true_labels[1,:num_points_per_cluster] = 1

for n in range(num_points_per_cluster):
    for k in range(K):
        theta_plus_noise = thetas[k]+np.random.random(3)*noise_scale-noise_scale/2

        #timeseries
        data_ts[n,:,k] = np.cos(theta_plus_noise)

        # eigenvector of cosinus matrix
        coh_map = np.outer(np.cos(theta_plus_noise),np.cos(theta_plus_noise))+np.outer(np.sin(theta_plus_noise),np.sin(theta_plus_noise))
        l,u = np.linalg.eig(coh_map)
        order = np.argsort(l)[::-1]
        data_real_projective_hyperplane[n,:,k] = u[:,order[0]]
        data_grassmann[n,:,:,k] = u[:,order[:q]]
        data_spsd[n,:,:,k] = u[:,order[:q]]*np.sqrt(l[order[:q]])[None]

        # eigenvector of complex coherence matrix
        coh_map_complex = np.outer(np.exp(1j*theta_plus_noise),np.exp(-1j*theta_plus_noise))
        l,u = np.linalg.eig(coh_map_complex)
        order = np.argsort(l)[::-1]
        data_complex_projective_hyperplane[n,:,k] = u[:,order[0]]

# concatenate data over the last dimension
data_ts = np.concatenate([data_ts[:,:,0],data_ts[:,:,1]],axis=0)
data_real_projective_hyperplane = np.concatenate([data_real_projective_hyperplane[:,:,0],data_real_projective_hyperplane[:,:,1]],axis=0)
data_complex_projective_hyperplane = np.concatenate([data_complex_projective_hyperplane[:,:,0],data_complex_projective_hyperplane[:,:,1]],axis=0)
data_grassmann = np.concatenate([data_grassmann[:,:,:,0],data_grassmann[:,:,:,1]],axis=0)
data_spsd = np.concatenate([data_spsd[:,:,:,0],data_spsd[:,:,:,1]],axis=0)

In [116]:
import pandas as pd
# Create a Pandas DataFrame to collect all results. Columns should be 'model', 'manifold', 'init', 'HMM', numpy/torch, 'NMI'
df = pd.DataFrame(columns=['model', 'manifold', 'init', 'HMM', 'numpy/torch', 'NMI'])

## K-means models

In [117]:
K=2 # Number of estimated components
max_iter = 10000 # Maximum number of iterations
init = '++' # Initialization method ('uniform' or '++')
tol = 1e-10 # Tolerance for the convergence criterion
num_repl = 2 # Number of estimates to choose the best from (different initializations)
from PCMM.phase_coherence_kmeans import *

The K-means models output the cluster centers, data partition vector, and objective function.

In [118]:
C_ls,X_part_ls,obj = least_squares_sign_flip(data_real_projective_hyperplane, K,max_iter=max_iter,init=init,tol=tol,num_repl=num_repl)
C_dc,X_part_dc,obj = diametrical_clustering(data_real_projective_hyperplane, K,max_iter=max_iter,init=init,tol=tol,num_repl=num_repl)
C_cdc,X_part_cdc,obj = diametrical_clustering(data_complex_projective_hyperplane, K,max_iter=max_iter,init=init,tol=tol,num_repl=num_repl)
C_gc,X_part_gc,obj = grassmann_clustering(data_grassmann, K,max_iter=max_iter,init=init,tol=tol,num_repl=num_repl)
C_wgc,X_part_wgc,obj = weighted_grassmann_clustering(data_spsd, K,max_iter=max_iter,init=init,tol=tol,num_repl=num_repl)

In [119]:
print('Least squares on sign-flipped leading eigenvectors: NMI='+str(calc_NMI(true_labels,np.eye(K)[X_part_ls].T)))
print('Diametrical clustering on real projective hyperplane: NMI='+str(calc_NMI(true_labels,np.eye(K)[X_part_dc].T)))
print('Diametrical clustering on complex projective hyperplane: NMI='+str(calc_NMI(true_labels,np.eye(K)[X_part_cdc].T)))
print('Grassmann clustering: NMI='+str(calc_NMI(true_labels,np.eye(K)[X_part_gc].T)))
print('Weighted Grassmann clustering: NMI='+str(calc_NMI(true_labels,np.eye(K)[X_part_wgc].T)))
df = pd.DataFrame({'model': ['Least squares on sign-flipped leading eigenvectors', 'Diametrical clustering', 'Complex diametrical clustering', 'Grassmann clustering', 'Weighted Grassmann clustering'],
                     'manifold': ['Real projective hyperplane', 'Real projective hyperplane', 'Complex projective hyperplane', 'Grassmann', 'Grassmann'],
                        'init': [init, init, init, init, init],
                        'HMM': [False, False, False, False, False],
                        'numpy/torch': ['numpy', 'numpy', 'numpy', 'numpy', 'numpy'],
                        'rank': [1,1,1,2,2],
                        'NMI': [calc_NMI(true_labels,np.eye(K)[X_part_ls].T), calc_NMI(true_labels,np.eye(K)[X_part_dc].T), calc_NMI(true_labels,np.eye(K)[X_part_cdc].T), calc_NMI(true_labels,np.eye(K)[X_part_gc].T), calc_NMI(true_labels,np.eye(K)[X_part_wgc].T)]})

Least squares on sign-flipped leading eigenvectors: NMI=0.17080061586805972
Diametrical clustering on real projective hyperplane: NMI=0.1433829549073654
Diametrical clustering on complex projective hyperplane: NMI=0.8314508234509516
Grassmann clustering: NMI=0.2289938158896587
Weighted Grassmann clustering: NMI=0.31698524782648857


## Mixture models, numpy estimation (EM)

### Instancing a mixture model
The implemented statistical distributions are:
- Watson (rank-1 component)
- Angular central Gaussian (ACG) (rank-r component)
- Matrix angular central Gaussian (MACG) (rank-r component)
- Singular Wishart (rank-r component)
- Gaussian (rank-r component)

Of these, Watson and ACG are for data on the (complex) projective hyperplane, i.e., the input data vectors should be normalized to unit norm. MACG is for data on the Grassmann manifold, i.e., the input data should be orthonormal matrices. Singular Wishart is for symmetric positive semidefinite matrices, i.e., the input data should be matrices corresponding to eigenvectors scaled by the square root of their eigenvalues. 

In our implementation, a mixture model first has to be instanced with some required inputs:
- K: number of components
- p: data dimensionality (e.g,. number of brain regions)
- q: number of orthonormal frames (only applicable to MACG and Singular Wishart)

Furthermore, the model instance hold the following optional inputs:
- rank [None]: The desired component rank (not applicable to Watson, which estimates a rank-1 component). Defaults to a full-rank model.
- complex [False]: Whether the distribution should be for complex data (only applicable to Watson and ACG)
- params [None]: Initial parameters for the model. Should be formatted the same way as for a model output.

### Estimating the model
For model estimation, we have made a training loop available, which has the following required inputs:
- model: The mixture model instance
- data: The input data as a numpy array of either shape (n, p) (projective hyperplane) or (n, p, q) (Grassmann or SPSD) depending on the model

The training loop has the following optional inputs:
- max_iter [10000]: Maximum number of iterations
- tol [1e-8]: Tolerance for convergence
- num_repl [1]: Number of restarts to choose the best model from
- init [None]: Initial parameters for the model. Options are 'uniform' for randomly initialized parameters, 'ls', 'dc', 'gc', and 'wgc' for K-means initialization as above, 'dc++', 'gc++', and 'wgc++' for K-means++ initialization, and 'ls_seg', 'dc_seg', 'gc_seg', and 'wgc_seg' for K-means followed by K single-component estimates. 

For initialization, note that 'dc' can be applied for Grassmann data and SPSD data, in that case, only the first column of the orthonormal matrices are used. Oppositely, 'gc' and 'wgc' cannot be used for projective hyperplane data.


In [120]:
# Parameters applying to all methods
max_iter = 10000 # Maximum number of iterations
tol = 1e-10 # Tolerance for the convergence criterion
params = None # Initial set of parameters
num_repl = 2

In [121]:
# Watson mixture model (real)
from PCMM.PCMMnumpy import Watson
model = Watson(K=K, p=p, complex=False, params=None)
init = 'dc' # initialization with diametrical clustering (which is itself seeded by dc++)
params, posterior, loglik = mixture_EM_loop(
    model=model, data=data_real_projective_hyperplane, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init
)
print('Watson mixture model (real): NMI='+str(calc_NMI(true_labels,posterior)))
df = pd.concat([df,pd.DataFrame({'model': 'Watson mixture', 'manifold': 'Real projective hyperplane', 'init': init, 'HMM': False, 'numpy/torch': 'numpy', 'rank': 1, 'NMI': [calc_NMI(true_labels,posterior)]})], ignore_index=True)

Running diametrical clustering initialization
Initializing mu based on the clustering centroid
Beginning EM loop


In the initial phase:   0%|          | 3/10000 [00:00<01:31, 108.94it/s]

Convergence towards tol: 6.80e-11:   0%|          | 40/10000 [00:00<01:51, 89.30it/s] 

Running diametrical clustering initialization
Initializing mu based on the clustering centroid
Beginning EM loop


Convergence towards tol: 9.41e-11:   0%|          | 39/10000 [00:00<02:11, 75.48it/s]

Watson mixture model (real): NMI=3.197703535880105e-05


In [122]:
# Watson mixture model (complex)
from PCMM.PCMMnumpy import Watson
model = Watson(K=K, p=p, complex=True, params=None)
init = 'dc' # initialization with diametrical clustering (which is itself seeded by dc++)
params, posterior, loglik = mixture_EM_loop(
    model=model, data=data_complex_projective_hyperplane, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init
)
print('Watson mixture model (complex): NMI='+str(calc_NMI(true_labels,posterior)))
df = pd.concat([df,pd.DataFrame({'model': 'Watson mixture', 'manifold': 'Complex projective hyperplane', 'init': init, 'HMM': False, 'numpy/torch': 'numpy', 'rank': 1, 'NMI': [calc_NMI(true_labels,posterior)]})], ignore_index=True)

Running diametrical clustering initialization
Initializing mu based on the clustering centroid
Beginning EM loop


Convergence towards tol: 5.99e-11:   0%|          | 11/10000 [00:00<01:22, 120.92it/s]

Running diametrical clustering initialization
Initializing mu based on the clustering centroid
Beginning EM loop


Convergence towards tol: 4.93e-11:   0%|          | 12/10000 [00:00<01:34, 105.93it/s]

Watson mixture model (complex): NMI=0.8725729124027487


In [123]:
# ACG mixture model (real)
rank=2
from PCMM.PCMMnumpy import ACG
model = ACG(K=K, p=p, rank=rank, complex=False, params=None)
init = 'dc' 
params, posterior, loglik = mixture_EM_loop(
    model=model, data=data_real_projective_hyperplane, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init
)
print('ACG mixture model (real): NMI='+str(calc_NMI(true_labels,posterior)))
df = pd.concat([df,pd.DataFrame({'model': 'ACG mixture', 'manifold': 'Real projective hyperplane', 'init': init, 'HMM': False, 'numpy/torch': 'numpy', 'rank': rank, 'NMI': [calc_NMI(true_labels,posterior)]})], ignore_index=True)

Running diametrical clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning EM loop


Convergence towards tol: 9.13e-11:   0%|          | 46/10000 [00:00<00:32, 307.32it/s]

Running diametrical clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning EM loop


Convergence towards tol: 9.13e-11:   0%|          | 47/10000 [00:00<00:48, 205.39it/s]


ACG mixture model (real): NMI=0.09293095204474758


In [124]:
# ACG mixture model (complex)
rank=2
from PCMM.PCMMnumpy import ACG
model = ACG(K=K, p=p, rank=rank, complex=True, params=None)
init = 'dc' 
params, posterior, loglik = mixture_EM_loop(
    model=model, data=data_complex_projective_hyperplane, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init
)
print('ACG mixture model (complex): NMI='+str(calc_NMI(true_labels,posterior)))
df = pd.concat([df,pd.DataFrame({'model': 'ACG mixture', 'manifold': 'Complex projective hyperplane', 'init': init, 'HMM': False, 'numpy/torch': 'numpy', 'rank': rank, 'NMI': [calc_NMI(true_labels,posterior)]})], ignore_index=True)

Running diametrical clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning EM loop


Convergence towards tol: 6.84e-11:   0%|          | 24/10000 [00:00<00:48, 207.54it/s]

Running diametrical clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning EM loop


Convergence towards tol: 7.33e-11:   0%|          | 24/10000 [00:00<00:58, 169.76it/s]

ACG mixture model (complex): NMI=0.7722618267731669


In [125]:
# MACG mixture model
from PCMM.PCMMnumpy import MACG
rank = 2
model = MACG(K=K, p=p, q=2, rank=rank, params=None)
init = 'gc'
params, posterior, loglik = mixture_EM_loop(
    model=model, data=data_grassmann, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init
)
print('MACG mixture model: NMI='+str(calc_NMI(true_labels,posterior)))
df = pd.concat([df,pd.DataFrame({'model': 'MACG mixture', 'manifold': 'Grassmann', 'init': init, 'HMM': False, 'numpy/torch': 'numpy', 'rank': rank, 'NMI': [calc_NMI(true_labels,posterior)]})], ignore_index=True)

Running grassmann clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning EM loop


Convergence towards tol: 8.73e-11:   0%|          | 28/10000 [00:02<06:02, 27.55it/s]

Running grassmann clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning EM loop


Convergence towards tol: 8.94e-11:   0%|          | 29/10000 [00:02<15:51, 10.48it/s]

MACG mixture model: NMI=7.672255690043816e-13


In [126]:
# Singular Wishart model
from PCMM.PCMMnumpy import SingularWishart
rank = 2
model = SingularWishart(K=K, p=p, q=2, rank=rank, params=None)
init = 'wgc'
params, posterior, loglik = mixture_EM_loop(
    model=model, data=data_spsd, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init
)
print('Singular Wishart model: NMI='+str(calc_NMI(true_labels,posterior)))
df = pd.concat([df,pd.DataFrame({'model': 'Singular Wishart', 'manifold': 'Grassmann', 'init': init, 'HMM': False, 'numpy/torch': 'numpy', 'rank': rank, 'NMI': [calc_NMI(true_labels,posterior)]})], ignore_index=True)

Running weighted grassmann clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning EM loop


In the initial phase:   0%|          | 6/10000 [00:00<01:20, 123.60it/s]

Convergence towards tol: 9.00e-11:   1%|          | 118/10000 [00:01<01:35, 102.94it/s]

Running weighted grassmann clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning EM loop


Convergence towards tol: 8.82e-11:   1%|▏         | 145/10000 [00:01<01:36, 102.48it/s]

Singular Wishart model: NMI=0.0902097707419764


## Mixture model estimation in PyTorch

Mixture estimation in PyTorch can be very fruitful for large datasets, both because of the possibility to run on GPUs and since EM estimation often requires intensive loops for each iteration. The PyTorch implementation is very similar to the numpy implementation, but with the following differences:

### Instancing a mixture model
A mixture model now has the option to also include a HMM and a samples_per_sequence parameter:
- HMM [False]: Whether the model should be a Hidden Markov Model
- samples_per_sequence [0]: Number of samples per sequence in the HMM to avoid temporal smoothing over sequences. If 0, it is assumed to be one sequence. If int, it is assumed to be N/samples_per_sequence sequences. Otherwise it can be a list. 

### Estimating the model
The training loop has the following additional optional inputs:
- LR [0.1]: Learning rate for the optimizer
- decrease_lr_on_plateau [False]: Whether the learning rate should decrease by a factor of 10 once when tol has been reached. 

In [127]:
# Parameters applying to all methods
max_iter = 10000 # Maximum number of iterations
tol = 1e-10 # Tolerance for the convergence criterion
params = None # Initial set of parameters
LR = 0.1 # Learning rate
num_repl = 2

In [128]:
# Watson mixture model (real)
from PCMM.PCMMtorch import Watson # Note the torch!
model = Watson(K=K, p=p, complex=False, params=None, HMM=False)
init = 'dc' 
params, posterior, loglik = mixture_torch_loop(
    model=model, data=data_real_projective_hyperplane, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init, LR=LR
)
print('Watson mixture model (real): NMI='+str(calc_NMI(true_labels,posterior.numpy())))
df = pd.concat([df,pd.DataFrame({'model': 'Watson mixture', 'manifold': 'Real projective hyperplane', 'init': init, 'HMM': False, 'numpy/torch': 'torch', 'rank': 1, 'NMI': [calc_NMI(true_labels,posterior.numpy())]})], ignore_index=True)

Running diametrical clustering initialization
Initializing mu based on the clustering centroid
Beginning numerical optimization loop


Convergence towards tol: 3.12e-06:   6%|▌         | 557/10000 [00:14<04:55, 31.96it/s]

Running diametrical clustering initialization
Initializing mu based on the clustering centroid
Beginning numerical optimization loop


Convergence towards tol: 2.56e-06:   6%|▋         | 649/10000 [00:18<04:31, 34.49it/s]

Watson mixture model (real): NMI=0.18600019908744703


In [129]:
# Watson mixture model (complex)
from PCMM.PCMMtorch import Watson
model = Watson(K=K, p=p, complex=True, params=None, HMM=False)
init = 'dc' # initialization with diametrical clustering (which is itself seeded by dc++)
params, posterior, loglik = mixture_torch_loop(
    model=model, data=data_complex_projective_hyperplane, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init, LR=LR
)
print('Watson mixture model (complex): NMI='+str(calc_NMI(true_labels,posterior.numpy())))
df = pd.concat([df,pd.DataFrame({'model': 'Watson mixture', 'manifold': 'Complex projective hyperplane', 'init': init, 'HMM': False, 'numpy/torch': 'torch', 'rank': 1, 'NMI': [calc_NMI(true_labels,posterior.numpy())]})], ignore_index=True)

Running diametrical clustering initialization
Initializing mu based on the clustering centroid
Beginning numerical optimization loop


Convergence towards tol: 1.17e-09:   3%|▎         | 256/10000 [00:06<04:16, 37.94it/s]

Running diametrical clustering initialization
Initializing mu based on the clustering centroid
Beginning numerical optimization loop


Convergence towards tol: 3.17e-09:   2%|▏         | 221/10000 [00:05<04:05, 39.88it/s]


Watson mixture model (complex): NMI=0.8725589362031273


In [130]:
# ACG mixture model (real)
rank=2
from PCMM.PCMMtorch import ACG
model = ACG(K=K, p=p, rank=rank, complex=False, params=None, HMM=False)
init = 'dc' 
params, posterior, loglik = mixture_torch_loop(
    model=model, data=data_real_projective_hyperplane, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init, LR=LR
)
print('ACG mixture model (real): NMI='+str(calc_NMI(true_labels,posterior.numpy())))
df = pd.concat([df,pd.DataFrame({'model': 'ACG mixture', 'manifold': 'Real projective hyperplane', 'init': init, 'HMM': False, 'numpy/torch': 'torch', 'rank': rank, 'NMI': [calc_NMI(true_labels,posterior.numpy())]})], ignore_index=True)

Running diametrical clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning numerical optimization loop


Convergence towards tol: 9.81e-11:  10%|█         | 1034/10000 [00:03<00:27, 328.35it/s]

Running diametrical clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning numerical optimization loop


Convergence towards tol: 9.82e-11:  10%|█         | 1007/10000 [00:06<01:00, 148.96it/s]

ACG mixture model (real): NMI=0.2181528271187925


In [131]:
# ACG mixture model (complex)
rank=2
from PCMM.PCMMtorch import ACG
model = ACG(K=K, p=p, rank=rank, complex=True, params=None, HMM=False)
init = 'dc' 
params, posterior, loglik = mixture_torch_loop(
    model=model, data=data_complex_projective_hyperplane, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init, LR=LR
)
print('ACG mixture model (complex): NMI='+str(calc_NMI(true_labels,posterior.numpy())))
df = pd.concat([df,pd.DataFrame({'model': 'ACG mixture', 'manifold': 'Complex projective hyperplane', 'init': init, 'HMM': False, 'numpy/torch': 'torch', 'rank': rank, 'NMI': [calc_NMI(true_labels,posterior.numpy())]})], ignore_index=True)

Running diametrical clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning numerical optimization loop


In the initial phase:   0%|          | 5/10000 [00:00<01:01, 161.69it/s]

Convergence towards tol: 2.03e-04:   0%|          | 19/10000 [00:00<00:48, 204.58it/s]

Running diametrical clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning numerical optimization loop


Convergence towards tol: 3.00e-04:   0%|          | 20/10000 [00:00<00:49, 201.05it/s]

ACG mixture model (complex): NMI=0.8105423853575899


### HMM
It's always a good idea to initialize HMMs from a learned mixture model. In this case, remember to set init='no'

In [132]:
# ACG mixture model (complex, HMM)
rank=2
from PCMM.PCMMtorch import ACG
import copy
model = ACG(K=K, p=p, rank=rank, complex=True, params=copy.deepcopy(params), HMM=True)
init = 'no' 
params, posterior, loglik = mixture_torch_loop(
    model=model, data=data_complex_projective_hyperplane, 
    tol=tol, max_iter=max_iter, num_repl=1, init=init, LR=LR
)
print('ACG mixture model (complex): NMI='+str(calc_NMI(true_labels,posterior.numpy())))
df = pd.concat([df,pd.DataFrame({'model': 'ACG mixture', 'manifold': 'Complex projective hyperplane', 'init': init, 'HMM': True, 'numpy/torch': 'torch', 'rank': rank, 'NMI': [calc_NMI(true_labels,posterior.numpy())]})], ignore_index=True)

Beginning numerical optimization loop


Convergence towards tol: 3.85e-07:   3%|▎         | 326/10000 [02:44<1:21:35,  1.98it/s]

ACG mixture model (complex): NMI=1.0


In [133]:
# MACG mixture model
from PCMM.PCMMtorch import MACG
rank = 2
model = MACG(K=K, p=p, q=2, rank=rank, params=None, HMM=False)
init = 'gc'
params, posterior, loglik = mixture_torch_loop(
    model=model, data=data_grassmann, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init, LR=LR
)
print('MACG mixture model: NMI='+str(calc_NMI(true_labels,posterior.numpy())))
df = pd.concat([df,pd.DataFrame({'model': 'MACG mixture', 'manifold': 'Grassmann', 'init': init, 'HMM': False, 'numpy/torch': 'torch', 'rank': rank, 'NMI': [calc_NMI(true_labels,posterior.numpy())]})], ignore_index=True)

Running grassmann clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning numerical optimization loop


  0%|          | 0/10000 [00:00<?, ?it/s]

Convergence towards tol: 9.67e-11:   5%|▍         | 482/10000 [00:05<01:44, 90.79it/s]

Running grassmann clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning numerical optimization loop


Convergence towards tol: 9.96e-11:   5%|▍         | 479/10000 [00:06<02:03, 77.35it/s]

MACG mixture model: NMI=0.2714402701708727


In [134]:
# Singular Wishart model
from PCMM.PCMMtorch import SingularWishart
rank = 2
model = SingularWishart(K=K, p=p, q=2, rank=rank, params=None, HMM=False)
init = 'wgc'
params, posterior, loglik = mixture_torch_loop(
    model=model, data=data_spsd, 
    tol=tol, max_iter=max_iter, num_repl=num_repl, init=init, LR=LR
)
print('Singular Wishart model: NMI='+str(calc_NMI(true_labels,posterior.numpy())))
df = pd.concat([df,pd.DataFrame({'model': 'Singular Wishart', 'manifold': 'Grassmann', 'init': init, 'HMM': False, 'numpy/torch': 'torch', 'rank': rank, 'NMI': [calc_NMI(true_labels,posterior.numpy())]})], ignore_index=True)

Running weighted grassmann clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning numerical optimization loop


Convergence towards tol: 2.67e-06:   2%|▏         | 195/10000 [00:01<00:54, 180.56it/s]

Running weighted grassmann clustering initialization
Initializing M based on a lowrank-svd of the input data partitioned acc to the clustering
Beginning numerical optimization loop


Convergence towards tol: 1.44e-04:   1%|          | 84/10000 [00:00<01:11, 138.71it/s]

Singular Wishart model: NMI=0.09490176895726506


See the results below. If interested, try changing the noise scale in the second cell to see what changes. The data generated is from a mixture of rank-1 true components, and thus, complex diametrical clustering and Watson mixtures perform well below, with no added advantage of ACG. All other models are unable to model consistent phase shifts well

In [135]:
df['NMI'] = df['NMI'].apply(lambda x: '{:.3f}'.format(x))
df

,model,manifold,init,HMM,numpy/torch,rank,NMI
0,Least squares on sign-flipped leading eigenvec...,Real projective hyperplane,++,False,numpy,1,0.171
1,Diametrical clustering,Real projective hyperplane,++,False,numpy,1,0.143
2,Complex diametrical clustering,Complex projective hyperplane,++,False,numpy,1,0.831
3,Grassmann clustering,Grassmann,++,False,numpy,2,0.229
4,Weighted Grassmann clustering,Grassmann,++,False,numpy,2,0.317
5,Watson mixture,Real projective hyperplane,dc,False,numpy,1,0.000
6,Watson mixture,Complex projective hyperplane,dc,False,numpy,1,0.873
7,ACG mixture,Real projective hyperplane,dc,False,numpy,2,0.093
8,ACG mixture,Complex projective hyperplane,dc,False,numpy,2,0.772
9,MACG mixture,Grassmann,gc,False,numpy,2,0.000
